# ナットの傷の検知


## Google Drive マウント

In [14]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/images/Nut"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/images/Nut


## ライブラリインストール・インポート

In [15]:
!pip install -q -U keras

import os
import glob
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
import seaborn

import keras
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D
from keras.layers import GlobalAveragePooling2D
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.pooling import MaxPool2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import plot_model

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
print("Tensorflow version: " + tf.__version__)

Tensorflow version: 2.2.0-rc2


## パラメータの指定


In [0]:
# エポック数
nb_epoch = 10

# バッチサイズ
nb_batch_size = 32

# トレーニング時に1つのイメージを1エポックで何回使用するか
nb_train_step_per_epoch = 8

# 検証時に1つのイメージを1エポックで何回使用するか
nb_val_step_per_epoch = 1

# ImageDataGeneratorによるAugmentationの設定
rescale = 1.0/255

zoom_range = 0.2
shear_range = 0.2

horizontal_flip = True
vertical_flip = True

width_shift_range = 0
height_shift_range = 0

brightness_range = [1.0, 1.0]
rotation_range = 90

# モデルへのinputイメージサイズとチャネル数
img_rows, img_cols = 224, 224
channels = 3


## データ生成

In [17]:
# ジェネレータの設定
datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = rescale,
    shear_range=shear_range,
    zoom_range=zoom_range,
    horizontal_flip=horizontal_flip,
    vertical_flip=vertical_flip,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    brightness_range=brightness_range,
    rotation_range=rotation_range,
    validation_split=0.1
    )


# 訓練セットを生成するジェネレータを作成
training_generator = datagen.flow_from_directory(
    os.path.abspath("data"),
    target_size=(img_rows, img_cols),
    batch_size=nb_batch_size,
    subset="training"
)

# 検証セットを生成するジェネレータを作成
validation_generator = datagen.flow_from_directory(
    os.path.abspath("data"),
    target_size=(img_rows, img_cols),
    batch_size=nb_batch_size,
    subset="validation"
)

nb_classes = len(training_generator.class_indices)

print(training_generator.labels)
print(validation_generator.labels)

Found 213 images belonging to 3 classes.
Found 22 images belonging to 3 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 1 1 1 1 2 2 2 2 2 2 2 2 2]


## モデルの作成

In [18]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels), kernel_initializer='he_normal'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())

model.add(Dense(nb_classes, activation='softmax'))

model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
dropout_5 (Dropout)          (None, 112, 112, 128)     0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 56, 56, 128)      

### モデルのコンパイル

In [0]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# 学習と結果の確認

### 学習

In [20]:
train_result = model.fit(
    training_generator,
    steps_per_epoch=len(training_generator)*nb_train_step_per_epoch,
    epochs=nb_epoch,
    verbose=1,
    validation_data=validation_generator
)

Epoch 1/10
56/56 [==============================] - 95s 2s/step - loss: 1.0887 - accuracy: 0.3867 - val_loss: 1.0480 - val_accuracy: 0.4091
Epoch 2/10
56/56 [==============================] - 49s 880ms/step - loss: 1.0535 - accuracy: 0.3985 - val_loss: 1.0498 - val_accuracy: 0.4091
Epoch 3/10
10/56 [====>.........................] - ETA: 41s - loss: 1.0563 - accuracy: 0.4228

KeyboardInterrupt: ignored

### 学習結果の確認

In [0]:
# １行２列のプロット画面を用意する
fig, axes = plt.subplots(1,2)
plt.rcParams['figure.figsize'] = (10.0, 10.0)
fig = plt.figure(figsize=(5,3),dpi=100)

# 左側図形にエポック毎の精度の推移を描画する
axes[0].set_title("Accuracy vs Epoch")
axes[0].plot(train_result.history["accuracy"], label="Training")
axes[0].plot(train_result.history["val_accuracy"], label="Validation")
axes[0].set_ylim(0,1)
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Accuracy")
axes[0].legend()

# 右側図形にエポック毎のロスの推移を描画する
axes[1].set_title("Loss vs Epoch")
axes[1].plot(train_result.history["loss"], label="Training")
axes[1].plot(train_result.history["val_loss"], label="Validation")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Loss")
axes[1].legend()

# 推論

In [0]:
test_files = glob.glob('test/*/*.jpg')
num_of_test_samples = len(test_files)

In [0]:
Y_pred = model.predict_generator(validation_generator, num_of_test_samples // nb_batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)

cm = confusion_matrix(validation_generator.classes, y_pred)

seaborn.heatmap(cm, cmap='Blues')

print('Confusion Matrix')
print(cm)

print('Classification Report')
target_names = ['clear', 'side_damaged', 'top_damaged']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))